In [27]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import clear_output


In [28]:
dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv') # training data
df_eval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv')
y_train = dftrain.pop('survived')
y_eval = df_eval.pop('survived')
# print(y_eval)
clear_output()

In [29]:
CATEGORICAL_COLUMNS = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck',
                       'embark_town', 'alone']
NUMERIC_COLUMNS = ['age', 'fare']

feature_columns = []
for i in CATEGORICAL_COLUMNS:
    vocabulary = dftrain[i].unique()
    feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(i,vocabulary))
for i in NUMERIC_COLUMNS:
    feature_columns.append(tf.feature_column.numeric_column(i,dtype=tf.float32))
print(feature_columns[7])

NumericColumn(key='age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)


In [30]:
def make_Input(dfinputs,dflabels,epochs=20,size=32,shuffle=True):
    def take():
        da = tf.data.Dataset.from_tensor_slices((dict(dfinputs), dflabels))
        if(shuffle):
            da.shuffle(1000)
        da = da.batch(size).repeat(epochs)
        return da
    return take

train_func = make_Input(dftrain,y_train)
eval_func = make_Input(df_eval,y_eval,epochs=1,shuffle=False)
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)
linear_est.train(train_func)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp34ahymv9', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. 

In [31]:
res = linear_est.evaluate(eval_func)
clear_output()
print(res['accuracy'])

0.7651515


In [34]:
prediction = list(linear_est.predict(eval_func))
clear_output()
for i in range(25):
    print(f"{round(prediction[i]['probabilities'][1])} => {y_eval[i]}")

0.0 => 0
0.0 => 0
1.0 => 1
1.0 => 1
0.0 => 1
1.0 => 1
1.0 => 0
0.0 => 0
1.0 => 0
1.0 => 1
1.0 => 1
0.0 => 0
1.0 => 1
0.0 => 0
1.0 => 1
1.0 => 0
0.0 => 0
1.0 => 1
0.0 => 1
0.0 => 0
0.0 => 0
0.0 => 0
1.0 => 1
0.0 => 0
1.0 => 0
